In [36]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.8 MB/s eta 0:

In [15]:
import gradio as gr

gr.Interface(lambda x:x, "microphone", "audio").launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [3]:
# Usual Libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn

# Librosa (the mother of audio files)
import librosa
import librosa.display
import IPython.display as ipd
import warnings
warnings.filterwarnings('ignore')

In [4]:
general_path = '/content/drive/MyDrive/Data'
print(list(os.listdir(f'{general_path}/genres_original/')))

['classical', 'metal', 'hiphop', 'rock', 'pop', 'country', 'disco', 'reggae', 'blues', 'jazz']


In [66]:
f = open('/content/drive/MyDrive/Data/genres_original/reggae/reggae.00035.wav', "r")
analyze_audio(f)

In [67]:
import gradio as gr
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_agg import FigureCanvasAgg

def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

def analyze_audio(file):
    # Load audio file
    y, sr = librosa.load(file.name)

    # Trim leading and trailing silence from audio signal
    y, _ = librosa.effects.trim(y)

    # Compute mel spectrogram
    n_fft = 2048
    hop_length = 512
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)
    S_DB = librosa.amplitude_to_db(S, ref=np.max)

    # Compute spectral centroid
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)

    # Compute zero crossing rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)

    # Plot mel spectrogram and spectral centroid
    fig, ax = plt.subplots(2, 1, figsize=(16, 10))
    librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='log', cmap='cool', ax=ax[0])
    ax[0].set_title("Mel Spectrogram", fontsize=23)
    frames = range(len(spectral_centroids))
    t = librosa.frames_to_time(frames, sr=sr, hop_length=hop_length)
    librosa.display.waveshow(y=y, sr=sr, alpha=0.4, color='#A300F9', ax=ax[1])
    ax[1].plot(t, normalize(spectral_centroids), color='#FFB100')
    ax[1].set_title("Spectral Centroid", fontsize=23)

    # Convert plot to image
    canvas = FigureCanvasAgg(fig)
    canvas.draw()
    mel_spec_arr = np.array(canvas.renderer.buffer_rgba())
    plt.close(fig)

    # Return image as numpy array
    return mel_spec_arr

inputs = gr.inputs.File(label="Upload audio file (WAV format)")
outputs = gr.outputs.Image(type="numpy", label="Mel Spectrogram and Spectral Centroid")
title = "Audio Analysis Dashboard"
description = "Upload an audio file (in WAV format) to analyze its mel spectrogram, spectral centroid, and zero crossing rate."
gr.Interface(fn=analyze_audio, inputs=inputs, outputs=outputs, title=title, description=description).launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>